# Estival/PyMC

In this notebook, we will build a BayesianCompartmentalModel, and calibrate it using PyMC

In [ ]:
# This is required for pymc parallel evaluation in notebooks
import multiprocessing as mp
import platform

if platform.system() != "Windows":
    mp.set_start_method("forkserver")

In [ ]:
import summer2
import numpy as np
import pandas as pd

In [ ]:
from summer2.extras import test_models

In [ ]:
m = test_models.sir()

In [ ]:
defp = m.get_default_parameters()
defp

In [ ]:
m.run({"contact_rate": 0.5, "recovery_rate": 0.4})
do_def = m.get_derived_outputs_df()
obs_clean = do_def["incidence"].iloc[0:50]
obs_noisy = obs_clean * np.exp(np.random.normal(0.0, 0.2, len(obs_clean)))
obs_clean.plot()
obs_noisy.plot(style=".")

In [ ]:
# The following imports are the 'building blocks' of estival models

# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [ ]:
# Specify a Truncated normal target with a free dispersion parameter
targets = [
    est.TruncatedNormalTarget(
        "incidence",
        obs_noisy,
        (0.0, np.inf),
        esp.UniformPrior("incidence_dispersion", (0.1, obs_noisy.max() * 0.1)),
    )
]

In [ ]:
# Uniform priors over our 2 model parameters
priors = [
    esp.UniformPrior("contact_rate", (0.01, 1.0)),
    esp.TruncNormalPrior("recovery_rate", 0.5, 0.2, (0.01, 1.0)),
]

In [ ]:
# The BayesianCompartmentalModel class is the primary entry point to all optimization and calibration
# methods in estival
# It takes a CompartmentalModel object, default parameters, priors, and targets
# The default parameters will be used as fixed values when no prior is specified for a given parameter

bcm = BayesianCompartmentalModel(m, defp, priors, targets)

In [ ]:
from estival.wrappers import pymc as epm

In [ ]:
import pymc as pm

In [ ]:
with pm.Model() as model:

    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)

    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    # pm.Deterministic("logp", model.logp(jacobian=False))

    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata = pm.sample(
        step=[pm.DEMetropolisZ(variables)],
        draws=4000,
        tune=0,
        cores=4,
        chains=4,
        mp_ctx=mp.get_start_method(),
    )

## Using arviz to examine outputs

In [ ]:
import arviz as az

In [ ]:
az.summary(idata)

In [ ]:
# Optional - select some subset out of the resulting trace - useful if
# you require additional burnin
# subset = idata.sel(draw=slice(500, None), groups="posterior")

In [ ]:
az.plot_trace(idata, figsize=(16, 3.2 * len(idata.posterior)), compact=False)
# , lines=[("m", {}, mtrue), ("c", {}, ctrue)]);

In [ ]:
az.plot_posterior(idata)

### Obtaining likelihood

It is frequently useful to examine the (log)likelihood values of the samples in addition to their distribution

In [ ]:
# likelihood_extras_for_idata will compute log likelihood, prior and posterior values,
# as well as invidual likelihood components for each target

from estival.sampling.tools import likelihood_extras_for_idata

In [ ]:
likelihood_df = likelihood_extras_for_idata(idata, bcm)

In [ ]:
likelihood_df

In [ ]:
# Examine the performance of chains over time

ldf_pivot = likelihood_df.reset_index(level="chain").pivot(columns=["chain"])

ldf_pivot["logposterior"].plot()

In [ ]:
# Sort this DataFrame by logposterior to obtain the MAP index
ldf_sorted = likelihood_df.sort_values(by="logposterior", ascending=False)

# Extract the parameters from the calibration samples
map_params = idata.posterior.to_dataframe().loc[ldf_sorted.index[0]].to_dict()

map_params

In [ ]:
# As you can see, this is exactly the value output from the original BCM
bcm.loglikelihood(**map_params), ldf_sorted.iloc[0]["loglikelihood"]

In [ ]:
# Run the model with these parameters
map_res = bcm.run(map_params)

In [ ]:
# ...and plot some results
variable = "incidence"

pd.Series(map_res.derived_outputs[variable]).plot(title=f"{variable} (MLE)")
bcm.targets[variable].data.plot(style=".")

#### Uncertainty sampling

In [ ]:
# Use the arviz extract method to obtain some samples, then convert to a DataFrame
sample_idata = az.extract(idata, num_samples=400)
samples_df = sample_idata.to_dataframe().drop(columns=["chain", "draw"])

In [ ]:
# We use estivals parallel tools to run the model evaluations

from estival.utils.parallel import map_parallel

In [ ]:
# Wrapper function captures our bcm from the main namespace to pass into map_parallel
# Using this idiom in closures/factory functions is typical
def run_sample(idx_sample):
    idx, params = idx_sample
    return idx, bcm.run(params)

In [ ]:
# Run the samples through our BCM using the above function
# map_parallel takes a function and an iterable as input

# We use 4 workers here, default is cpu_count/2 (assumes hyperthreading)
sample_res = map_parallel(run_sample, samples_df.iterrows(), n_workers=4)

In [ ]:
# We'll use xarray for this step; aside from computing things very quickly, it's useful
# to persist the run results to netcdf/zarr etc

import xarray as xr

In [ ]:
# Build a DataArray out of our results, then assign coords for indexing
xres = xr.DataArray(
    np.stack([r.derived_outputs for idx, r in sample_res]),
    dims=["sample", "time", "variable"],
)
xres = xres.assign_coords(
    sample=sample_idata.coords["sample"],
    time=map_res.derived_outputs.index,
    variable=map_res.derived_outputs.columns,
)

In [ ]:
# Set some quantiles to calculate
quantiles = (0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99)

# Generate a new DataArray containing the quantiles
xquantiles = xres.quantile(quantiles, dim=["sample"])

In [ ]:
# Extract these values to a pandas DataFrame for ease of plotting

uncertainty_df = (
    xquantiles.to_dataframe(name="value")
    .reset_index()
    .set_index("time")
    .pivot(columns=("variable", "quantile"))["value"]
)

In [ ]:
variable = "incidence"

fig = uncertainty_df[variable].plot(title=variable, alpha=0.7)
pd.Series(map_res.derived_outputs[variable]).plot(style="--")
bcm.targets[variable].data.plot(style=".", color="black", ms=3, alpha=0.8)